<a href="https://colab.research.google.com/github/vib-hor/Colab-Archive/blob/main/ShapeDetection/ShapeRecognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hough Transform 

Hough Transform is an algorithmic approach to identify shapes in an image using basic image transformations. It is usually used to identify shapes that can be represented by few parameters like lines(m,c), circles(a,b,r), etc. The basic flow of hough transform is shown in the image:


![picture](https://github.com/vib-hor/Colab-Archive/blob/main/ShapeDetection/images/HoughFlow.png?raw=true)



Hough Transform is used to connect disjoint edge points in an image, so preprocessing is done to extract features using edge detection algorithms (like Canny, Sobel, Laplacian) and then using hough transform the edges  are grouped to highlight shapes. The edge detection algorithm gives a set of coordinates $(x_i,y_i)$ which are then used by the hough transform algorithm depending on the target shape to be identified.

In [ ]:
#import all the dependencies
import cv2
import numpy as np
from matplotlib import pyplot as plt
#to use cv_imshow (opencv's cv2.imshow doesn't work with colab)  
#alternatively use plt.imshow() 
from google.colab.patches import cv2_imshow

In [ ]:
#clone Repo to import images
!git clone https://github.com/vib-hor/Colab-Archive.git

# Hough Line Transform
---
Any line in a two-dimensional plane can be represented by the equation $y=mx+c$, where $m$ is the slope and $c$ is the $y$ intercept of the line. The Hough space is defined as a two-dimensional plane with the horizontal axis representing the slope and the vertical axis representing the intercept of the line on the edge image.
Any line on the edge image $(y=mx+c)$ produces a point on the Hough Space since a line is characterized by its slope $m$ and intercept $c$. On the other hand, an edge point $(x_i,y_i)$ on the edge image can have an infinite number of lines pass through it. Therefore, an edge point produces a line in the Hough Space in the form of $c=−mx_i+y_i$. In the Hough Transform algorithm, the Hough Space is used to determine whether a line exists in the edge image.
![HoughSpace.png](https://github.com/vib-hor/Colab-Archive/blob/main/ShapeDetection/images/HoughSpace.png?raw=true)

## Algorithm:


>Step1: Create a Quantized Hough Space $m$-$c$

>Step2: Create an Accumulator array $A(m,c)$ and initialize it to 0 

>Step3: For each point $(x_i,y_i)$ on line $c = -m x_i +y_i$
,do $A(m,c)=A(m,c)+1$

>Step4: Look for the maxima in Acumulator array.


![Algorithm](https://github.com/vib-hor/Colab-Archive/blob/main/ShapeDetection/images/Algorithm.png?raw=true)

### Pratical Implementation 
For the algorithm the parameter chosen were m and c, where m belongs to $(-\infty ,\infty)$, this requires a larger accumulator and uses more memory.
Instead of using the $y=mx+c$ the polor representation is used $\rho=xcos(\theta)+ysin(\theta)$, where $\rho$ length (finite) of normal to the line from origin and $\theta$ (ranging from $[0,\pi)$ ) is the angle made by line with x-axis. Hence the new hough space will be defined by $\rho$-$\theta$ space.

In [ ]:
#read the image
lineImage = cv2.imread('Colab-Archive/ShapeDetection/images/lines.jpg')
#Convert to grayscale
lineGray = cv2.cvtColor(lineImage, cv2.COLOR_BGR2GRAY)

lineEdge = cv2.Canny(lineGray, 50, 400)


# blur = cv2.medianBlur(mag_im, 3)
line = cv2.HoughLines(lineEdge, 1, np.pi/180, 200)
lineOutput = lineImage.copy()

for lines in line:
    rho = lines[0, 0]
    theta = lines[0, 1]

    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a*rho
    y0 = b*rho
    x1 = int(x0 + 1000*(-b))
    y1 = int(y0 + 1000*(a))
    x2 = int(x0 - 1000*(-b))
    y2 = int(y0 - 1000*(a))

    lineOutput = cv2.line(lineOutput, (x1, y1), (x2, y2), (0, 0, 255), thickness=3)

plt.imshow(lineGray,cmap="gray")
plt.show()

plt.imshow(lineEdge,cmap="gray")
plt.show()

plt.imshow(lineOutput,cmap="gray")
plt.show()


# Hough Cicle Transform

Similar to the Hough Line transform the Hough Circle Transform is done by defining hough space based on the equation of the circle: $(x_i - a)^2+(y_i-b)^2=r^2$, where $(a,b)$ are the coordinates of the center of the circle and $r$ is the radius. So, for a known radius, the Hough Space can be defined by $a$-$b$ space. Any point on the circle in the image will be a circle in Hough Space and the intersection of all the circles formed by available image points will give the parameters of the circle.
Similar to the Hough Line Transform the polar equation of circle $x = a + rcos(\theta), y = b+rsin(\theta)$ is used for the practical implementation. 

In [ ]:
#read the image
image = cv2.imread('Colab-Archive/ShapeDetection/images/circles.jpg')
#convert to grayscale 
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#Add blur with a mask of 3/5
blur = cv2.medianBlur(gray, 3)
#Hough Circle Transform
edges = cv2.Canny(blur, 50, 200)
circles = cv2.HoughCircles(blur, cv2.HOUGH_GRADIENT,1.5,10)
# circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, 1, 10)
# circles = np.uint16(np.around(circles))
output = image.copy()
#Draw circles on image
for i in circles[0,:]:
    cv2.circle(output,(i[0], i[1]), i[2], (255, 0, 0), 2)
    cv2.circle(output, (i[0], i[1]), 2, (0, 255, 0), 5)
plt.imshow(image, cmap='gray')
plt.show()
plt.imshow(edges, cmap='gray')
plt.show()
plt.imshow(output, cmap='gray')
plt.show()